In [28]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper


In [29]:
wiki_wrapper= WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki=WikipediaQueryRun(api_wrapper=wiki_wrapper)

In [30]:
# custom tool
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
import bs4

loader= WebBaseLoader("https://docs.smith.langchain.com/")
docs= loader.load()
documents= RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectorDb= FAISS.from_documents(documents,OpenAIEmbeddings())
retriver= vectorDb.as_retriever()


In [31]:
from langchain.tools.retriever import create_retriever_tool
retriver_tool= create_retriever_tool(retriver, "langsmith_search",
                      "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!")

In [32]:
retriver_tool.name

'langsmith_search'

In [33]:
# Arxiv Tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper= ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)

In [34]:
tools=[wiki,arxiv, retriver_tool]

In [35]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\ammar\\OneDrive\\Desktop\\Git\\Langchain-GenAI\\venv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200, arxiv_result=<class 'arxiv.Result'>)),
 Tool(name='langsmith_search', description='Search for information about LangSmith. For any questions about LangSmith, you must use this tool!', args_schema=<class 'langchain_core.tools.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000024A7AD9FCE0>, retriever=VectorStoreRetriever(tags=['FAISS

In [37]:
#  Agents
from langchain.agents import create_openai_tools_agent
from langchain_openai import ChatOpenAI
from langchain import hub

# can use chatprompttemplate instead of hub
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages
llm = ChatOpenAI(model="gpt-3.5-turbo")
agent= create_openai_tools_agent(llm,tools,prompt)

In [41]:
# Agent Executive

from langchain.agents import AgentExecutor
agent_executor= AgentExecutor(agent=agent,tools=tools,verbose=True)


In [42]:
agent_executor.invoke({"input":"Tell me about langsmith"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'LangSmith'}`


Get started with LangSmith | 🦜️🛠️ LangSmith

Skip to main contentGo to API DocsSearchGo to AppQuick startTutorialsHow-to guidesConceptsReferencePricingSelf-hostingLangGraph CloudQuick startOn this pageGet started with LangSmithLangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!1. Install LangSmith​PythonTypeScriptpip install -U langsmithyarn add langchain langsmith2. Create an API key​To create an API key head to the Settings page. Then click Create API Key.3. Set up your environment​Shellexport LANGCHAIN_TRACING_V2=trueexport LANGCHAIN_API_KEY=<your-api-key># The below examples use the OpenAI API, though it's not necessary in generalexport OPENAI_API_KEY=<your-openai-api-key>4. Log your first tr

{'input': 'Tell me about langsmith',
 'output': 'LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. LangSmith works on its own and does not require the use of LangChain. To get started with LangSmith, you can follow these steps:\n\n1. Install LangSmith:\n   - Python: `pip install -U langsmith`\n   - TypeScript: `yarn add langchain langsmith`\n\n2. Create an API key:\n   - Head to the Settings page and click on Create API Key.\n\n3. Set up your environment:\n   - Export the necessary environment variables like `LANGCHAIN_TRACING_V2`, `LANGCHAIN_API_KEY`, and `OPENAI_API_KEY`.\n\n4. Log your first trace:\n   - LangSmith provides multiple ways to log traces. You can use the provided code examples to log your first trace.\n\nYou can learn more about LangSmith and its functionalities in the LangSmith documentation and tutorials provided on their website.'}

In [43]:
agent_executor.invoke({"input":"Tell me about ML"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `Machine Learning`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and genMachine learning (ML) is a field of study in artificial intelligence that focuses on the development and study of statistical algorithms capable of learning from data and generating insights or predictions. It involves creating models and systems that can improve their performance over time based on the input data. ML algorithms are used in various applications, including data analysis, pattern recognition, natural language processing, and computer vision.

> Finished chain.


{'input': 'Tell me about ML',
 'output': 'Machine learning (ML) is a field of study in artificial intelligence that focuses on the development and study of statistical algorithms capable of learning from data and generating insights or predictions. It involves creating models and systems that can improve their performance over time based on the input data. ML algorithms are used in various applications, including data analysis, pattern recognition, natural language processing, and computer vision.'}

In [44]:
agent_executor.invoke({"input":"whats the paper 1605.08386"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1605.08386'}`


Published: 2016-05-26
Title: Heat-bath random walks with Markov bases
Authors: Caprice Stanley, Tobias Windisch
Summary: Graphs on lattice points are studied whose edges come from a finite set of
alloThe paper with the identifier 1605.08386 is titled "Heat-bath random walks with Markov bases" by Caprice Stanley and Tobias Windisch. It was published on May 26, 2016. The paper discusses the study of graphs on lattice points where the edges are derived from a finite set of allocations.

> Finished chain.


{'input': 'whats the paper 1605.08386',
 'output': 'The paper with the identifier 1605.08386 is titled "Heat-bath random walks with Markov bases" by Caprice Stanley and Tobias Windisch. It was published on May 26, 2016. The paper discusses the study of graphs on lattice points where the edges are derived from a finite set of allocations.'}